# Vlaamse parlement API

Het Vlaamse Parlement stelt de data uit de parlementaire databank ook ter beschikking via een API: https://www.vlaamsparlement.be/nl/parlementair-werk/dossiers/dossiers/open-data en http://ws.vlpar.be/e/opendata/api/.

# Setting up

In [1]:
# show all outputs of cell, not merely of last line (i.e. default of Jupyter Notebook)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd

import csv
import pickle

from collections import defaultdict

import requests

from datetime import datetime
import locale # to allow date parsing for dates in Dutch

from collections import Counter

import matplotlib.pyplot as plt
import plotly.express as px # for scatterplots

In [3]:
# # Set the locale to Dutch
# locale.setlocale(locale.LC_ALL, 'nl_NL')

In [4]:
# Dictionary mapping parties to colors
# # Use colours obtained from website: manual insertion or reading in dict
party_colors = {'Groen': '83de62',
    'Onafhankelijke': '787878',
    'Vooruit': 'FF2900',
    'Vlaams Belang': 'ffe500',
    'cd&v': 'f5822a',
    'N-VA': 'ffac12',
    'Open Vld': '003d6d',
    'PVDA': 'AA050E'
 }

In [5]:
# Obtain string of current date
today_str = datetime.now().strftime("%Y-%m-%d")
today_str 

'2023-12-18'

In [6]:
# Set base_url of api
base_url = "https://ws.vlpar.be/e/opendata"

The webpage of the API shows some interesting fields:
* `/stats/{commId}/{zj}`: statistieken voor commissie per zittingsjaar
* `/vv/huidige`: Lijst van huidige Vlaamse volksvertegenwoordigers
* `/vv/gewezen`: Lijst van gewezen Vlaamse volksvertegenwoordigers
* `/vv/{persoonId}` Detailgegevens volksvertegenwoordiger
* `/comm/huidige` Commissies van de huidige legislatuur
* `/comm/{commId}` Samenstelling commissie
* `/verg/vorige` Lijst van vorige vergaderingen voor periode
* `/verg/zoek/datums` Lijst van vorige vergaderingen (beperkte data) binnen een zekere periode

# Read in data

First we redefine the function to obtain an endpoint. 

In [7]:
def get_endpoint(endpoint: str):
    """
    Return data available at inserted endpoint
    """
    # Make the GET request
    response = requests.get(f"{base_url}{endpoint}")

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()  # Parse JSON response
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    return data

Then we load in the relevant data, as obtained earlier:
* `overall_attendance_dict`: dictionary with for each commission an overview of all meetings and how it was attended
* `fracties_dict`: overview of the current fractions with their members and id's
* `parlementsleden_all_dict`: overview of all current members of parliament, their party and their ID
* `commissions_overview_df`: dateframe of all commissions, their permanent members and amount of meetings

In [8]:
# # Load data of each meeting for each commission
# with open(f'../data/vergaderingen_commissies/overall_attendance_dict_2023-12-09.pkl', 'rb') as file:
#     overall_attendance_dict = pickle.load(file)

# # Inspect results
# overall_attendance_dict.keys()
# overall_attendance_dict['Commissie voor Algemeen Beleid, Financiën, Begroting en Justitie']

In [9]:
# Load information about fracties
with open(f'../data/fracties.pkl', 'rb') as file:
    fracties_dict = pickle.load(file)

with open(f'../data/parlementsleden.pkl', 'rb') as file:
    parlementsleden_all_dict = pickle.load(file)

# Inspect results
fracties_dict
parlementsleden_all_dict

{'Groen': [['Staf Aerts', 4544],
  ['Meyrem Almaci', 4381],
  ['Imade Annouri', 4052],
  ['Stijn Bex', 4385],
  ['Johan Danen', 4069],
  ['Ann De Martelaer', 1781],
  ['Celia Groothedde', 4398],
  ['Elisabeth Meuleman', 3173],
  ['An Moerenhout', 4058],
  ['Björn Rzoska', 3920],
  ['Mieke Schauvliege', 4409],
  ['Chris Steenwegen', 4414],
  ['Tine Van den Brande', 4419],
  ['Jeremie Vaneeckhout', 4426]],
 'Onafhankelijke': [['Els Ampe', 4382], ['Sihame El Kaouakibi', 4396]],
 'Vooruit': [['Hannes Anaf', 4383],
  ['Kurt De Loor', 2791],
  ['Hannelore Goeman', 4397],
  ['Annick Lambrecht', 4402],
  ['Els Robeyns', 3221],
  ['Katia Segers', 4087],
  ['Bruno Tobback', 1882],
  ['Steve Vandenberghe', 4107],
  ['Freya Van den Bossche', 3458],
  ['Ludwig Vandenhove', 4423],
  ['Thijs Verbeurgt', 4583],
  ['Maxim Veys', 4428]],
 'Vlaams Belang': [['Roosmarijn Beckers', 4384],
  ['Adeline Blancquaert', 4386],
  ['Filip Brusselmans', 4387],
  ['Yves Buysse', 4388],
  ['Bart Claes', 4390],
  ['Jo

{4544: ['Staf Aerts', 'Groen'],
 4381: ['Meyrem Almaci', 'Groen'],
 4382: ['Els Ampe', 'Onafhankelijke'],
 4383: ['Hannes Anaf', 'Vooruit'],
 4052: ['Imade Annouri', 'Groen'],
 4384: ['Roosmarijn Beckers', 'Vlaams Belang'],
 4385: ['Stijn Bex', 'Groen'],
 4386: ['Adeline Blancquaert', 'Vlaams Belang'],
 3430: ['Robrecht Bothuyne', 'cd&v'],
 3426: ['Karin Brouwers', 'cd&v'],
 4387: ['Filip Brusselmans', 'Vlaams Belang'],
 4388: ['Yves Buysse', 'Vlaams Belang'],
 4389: ['Allessia Claes', 'N-VA'],
 4390: ['Bart Claes', 'Vlaams Belang'],
 4444: ['Arnout Coel', 'N-VA'],
 4446: ['Steven Coenegrachts', 'Open Vld'],
 4068: ['Cathy Coudyser', 'N-VA'],
 4069: ['Johan Danen', 'Groen'],
 4070: ['Koen Daniëls', 'N-VA'],
 2838: ['Johan Deckmyn', 'Vlaams Belang'],
 4391: ['Inez De Coninck', 'N-VA'],
 3416: ['Jean-Jacques De Gucht', 'Open Vld'],
 2791: ['Kurt De Loor', 'Vooruit'],
 1781: ['Ann De Martelaer', 'Groen'],
 4432: ['Immanuel De Reuse', 'Vlaams Belang'],
 2793: ['Annick De Ridder', 'N-VA'],


In [10]:
# Load information about fracties
with open(f'../data/commissions_overview_df_2023-12-12.pkl', 'rb') as file:
    commissions_overview_df = pickle.load(file)
    
# Inspect results
commissions_overview_df

,commissie.id,commissie.titel,commissie.link,voorzitter,eerste ondervoorzitter,tweede ondervoorzitter,derde ondervoorzitter,vierde ondervoorzitter,vaste leden,plaatsvervangende leden,...,verontschuldigd_count_alle,aanwezig_count_vaste,afwezig_count_vaste,verontschuldigd_count_vaste,Gemiddelde aantal aanwezig alle leden,Gemiddelde aantal afwezig alle leden,Gemiddelde aantal verontschuldigd alle leden,Gemiddelde aantal aanwezig vaste leden,Gemiddelde aantal afwezig vaste leden,Gemiddelde aantal verontschuldigd vaste leden
0,1622680,Commissie Vlaams Energie- en Klimaatplan,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Andries Gryffroy],[Bruno Tobback],[Robrecht Bothuyne],[],None,"[Andries Gryffroy, Bruno Tobback, Robrecht Bot...","[Adeline Blancquaert, Bart Claes, Inez De Coni...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
1,1332937,"Commissie voor Algemeen Beleid, Financiën, Beg...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Steven Vandeput],[Willem-Frederik Schiltz],[Arnout Coel],[],None,"[Steven Vandeput, Willem-Frederik Schiltz, Arn...","[Filip Brusselmans, Yves Buysse, Stijn De Roo,...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
2,1332966,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Kris Van Dijck],[Sam Van Rooy],[Bart Tommelein],[],None,"[Kris Van Dijck, Sam Van Rooy, Bart Tommelein,...","[Adeline Blancquaert, Guy D'haeseleer, Stephan...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
3,1332996,Commissie voor Brussel en de Vlaamse Rand en D...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Mercedes Van Volcem],[Karl Vanlouwe],[Klaas Slootmans],[],None,"[Mercedes Van Volcem, Karl Vanlouwe, Klaas Slo...","[Meyrem Almaci, Karin Brouwers, Filip Brusselm...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
4,1333030,"Commissie voor Buitenlands Beleid, Europese Aa...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Cathy Coudyser],[Johan Deckmyn],[Tine Van den Brande],[],None,"[Cathy Coudyser, Johan Deckmyn, Tine Van den B...","[Adeline Blancquaert, Jean-Jacques De Gucht, M...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
5,1333053,"Commissie voor Cultuur, Jeugd, Sport en Media",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Meyrem Almaci],[Marius Meremans],[Karin Brouwers],[],None,"[Meyrem Almaci, Marius Meremans, Karin Brouwer...","[Immanuel De Reuse, Maaike De Rudder, Andries ...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
6,1333088,"Commissie voor Economie, Werk, Sociale Economi...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Robrecht Bothuyne],[Imade Annouri],[Andries Gryffroy],[],None,"[Robrecht Bothuyne, Imade Annouri, Andries Gry...","[Steven Coenegrachts, Johan Danen, Annick De R...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
7,1333137,"Commissie voor Landbouw, Visserij en Plattelan...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Dochy],[Joris Nachtergaele],[Emmily Talpe],[],None,"[Bart Dochy, Joris Nachtergaele, Emmily Talpe,...","[Roosmarijn Beckers, Allessia Claes, Steven Co...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
8,1333177,"Commissie voor Leefmilieu, Natuur, Ruimtelijke...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Gwenny De Vroe],[Bruno Tobback],[Inez De Coninck],[],None,"[Gwenny De Vroe, Bruno Tobback, Inez De Coninc...","[Hannes Anaf, Robrecht Bothuyne, Johan Danen, ...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
9,1333200,Commissie voor Mobiliteit en Openbare Werken,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Claes],[Martine Fournier],[Els Robeyns],[],None,"[Bart Claes, Martine Fournier, Els Robeyns, Im...","[Roosmarijn Beckers, Karin Brouwers, Arnout Co...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Read in all meetings and attendance as df
meetings_all_commissions_df = pd.read_pickle('../data/meetings_all_commissions_df_2023-12-12.pkl')

In [12]:
# Inspect results
meetings_all_commissions_df.shape
meetings_all_commissions_df.head()

meetings_jeugd = meetings_all_commissions_df[meetings_all_commissions_df["commissie.titel"] == "Commissie voor Cultuur, Jeugd, Sport en Media"]
# meetings_jeugd
meetings_jeugd.shape

# overall_attendance_dict["Commissie voor Cultuur, Jeugd, Sport en Media"].shape

(170, 14)

,AANWEZIG_vast,AFWEZIG_vast,VERONTSCHULDIGD_vast,commissie.titel,Datum vergadering,AANWEZIG,AFWEZIG,VERONTSCHULDIGD,Aantal aanwezig alle leden,Aantal afwezig alle leden,Aantal verontschuldigd alle leden,Aantal aanwezig vaste leden,Aantal afwezig vaste leden,Aantal verontschuldigd vaste leden
Vergadering 1761823,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-09-19,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...",8.0,4.0,4.0,7.0,4.0,4.0
Vergadering 1763264,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-09-26,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...",9.0,3.0,4.0,8.0,3.0,4.0
Vergadering 1763750,"[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Björn Rzoska', 'id': 3920, 'Fractie...","[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-10-03,"[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Björn Rzoska', 'id': 3920, 'Fractie...","[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...",9.0,4.0,5.0,7.0,4.0,4.0
Vergadering 1766893,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-10-10,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Stijn De Roo', 'id': 4525, 'Fractie...",11.0,3.0,6.0,8.0,3.0,4.0
Vergadering 1770223,"[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Andries Gryffroy', 'id': 4077, 'Fra...","[{'Naam': 'Celia Groothedde', 'id': 4398, 'Fra...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-10-17,"[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Andries Gryffroy', 'id': 4077, 'Fra...","[{'Naam': 'Johan Deckmyn', 'id': 2838, 'Fracti...",13.0,2.0,5.0,10.0,2.0,3.0


(18, 14)

## Obtain attendance statistics

In [13]:
#Set values for further testing
commission_value = "Alle commissies"
# commission_value = "Commissie voor Cultuur, Jeugd, Sport en Media"

# Filter i) the DataFrame with all commission meetings based on the commission dropdown value and
# ii) the overview dataframe based on the commission dropdown value
if commission_value == "Alle commissies":
    meetings_all_commissions_filtered_df = meetings_all_commissions_df
    commissions_overview_filtered_df = commissions_overview_df
else:
    meetings_all_commissions_filtered_df = meetings_all_commissions_df[
        meetings_all_commissions_df['commissie.titel'] == commission_value
    ]
    commissions_overview_filtered_df = commissions_overview_df[
        commissions_overview_df['commissie.titel'] == commission_value
    ]


In [14]:
# Inspect results
commissions_overview_filtered_df
meetings_all_commissions_filtered_df

,commissie.id,commissie.titel,commissie.link,voorzitter,eerste ondervoorzitter,tweede ondervoorzitter,derde ondervoorzitter,vierde ondervoorzitter,vaste leden,plaatsvervangende leden,...,verontschuldigd_count_alle,aanwezig_count_vaste,afwezig_count_vaste,verontschuldigd_count_vaste,Gemiddelde aantal aanwezig alle leden,Gemiddelde aantal afwezig alle leden,Gemiddelde aantal verontschuldigd alle leden,Gemiddelde aantal aanwezig vaste leden,Gemiddelde aantal afwezig vaste leden,Gemiddelde aantal verontschuldigd vaste leden
0,1622680,Commissie Vlaams Energie- en Klimaatplan,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Andries Gryffroy],[Bruno Tobback],[Robrecht Bothuyne],[],None,"[Andries Gryffroy, Bruno Tobback, Robrecht Bot...","[Adeline Blancquaert, Bart Claes, Inez De Coni...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
1,1332937,"Commissie voor Algemeen Beleid, Financiën, Beg...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Steven Vandeput],[Willem-Frederik Schiltz],[Arnout Coel],[],None,"[Steven Vandeput, Willem-Frederik Schiltz, Arn...","[Filip Brusselmans, Yves Buysse, Stijn De Roo,...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
2,1332966,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Kris Van Dijck],[Sam Van Rooy],[Bart Tommelein],[],None,"[Kris Van Dijck, Sam Van Rooy, Bart Tommelein,...","[Adeline Blancquaert, Guy D'haeseleer, Stephan...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
3,1332996,Commissie voor Brussel en de Vlaamse Rand en D...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Mercedes Van Volcem],[Karl Vanlouwe],[Klaas Slootmans],[],None,"[Mercedes Van Volcem, Karl Vanlouwe, Klaas Slo...","[Meyrem Almaci, Karin Brouwers, Filip Brusselm...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
4,1333030,"Commissie voor Buitenlands Beleid, Europese Aa...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Cathy Coudyser],[Johan Deckmyn],[Tine Van den Brande],[],None,"[Cathy Coudyser, Johan Deckmyn, Tine Van den B...","[Adeline Blancquaert, Jean-Jacques De Gucht, M...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
5,1333053,"Commissie voor Cultuur, Jeugd, Sport en Media",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Meyrem Almaci],[Marius Meremans],[Karin Brouwers],[],None,"[Meyrem Almaci, Marius Meremans, Karin Brouwer...","[Immanuel De Reuse, Maaike De Rudder, Andries ...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
6,1333088,"Commissie voor Economie, Werk, Sociale Economi...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Robrecht Bothuyne],[Imade Annouri],[Andries Gryffroy],[],None,"[Robrecht Bothuyne, Imade Annouri, Andries Gry...","[Steven Coenegrachts, Johan Danen, Annick De R...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
7,1333137,"Commissie voor Landbouw, Visserij en Plattelan...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Dochy],[Joris Nachtergaele],[Emmily Talpe],[],None,"[Bart Dochy, Joris Nachtergaele, Emmily Talpe,...","[Roosmarijn Beckers, Allessia Claes, Steven Co...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
8,1333177,"Commissie voor Leefmilieu, Natuur, Ruimtelijke...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Gwenny De Vroe],[Bruno Tobback],[Inez De Coninck],[],None,"[Gwenny De Vroe, Bruno Tobback, Inez De Coninc...","[Hannes Anaf, Robrecht Bothuyne, Johan Danen, ...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
9,1333200,Commissie voor Mobiliteit en Openbare Werken,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Claes],[Martine Fournier],[Els Robeyns],[],None,"[Bart Claes, Martine Fournier, Els Robeyns, Im...","[Roosmarijn Beckers, Karin Brouwers, Arnout Co...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN


,AANWEZIG_vast,AFWEZIG_vast,VERONTSCHULDIGD_vast,commissie.titel,Datum vergadering,AANWEZIG,AFWEZIG,VERONTSCHULDIGD,Aantal aanwezig alle leden,Aantal afwezig alle leden,Aantal verontschuldigd alle leden,Aantal aanwezig vaste leden,Aantal afwezig vaste leden,Aantal verontschuldigd vaste leden
Vergadering 1761823,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-09-19,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...",8.0,4.0,4.0,7.0,4.0,4.0
Vergadering 1763264,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-09-26,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...",9.0,3.0,4.0,8.0,3.0,4.0
Vergadering 1763750,"[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Björn Rzoska', 'id': 3920, 'Fractie...","[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-10-03,"[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Björn Rzoska', 'id': 3920, 'Fractie...","[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...",9.0,4.0,5.0,7.0,4.0,4.0
Vergadering 1766893,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Maaike De Vreese', 'id': 4394, 'Fra...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-10-10,"[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...","[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Stijn De Roo', 'id': 4525, 'Fractie...",11.0,3.0,6.0,8.0,3.0,4.0
Vergadering 1770223,"[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Andries Gryffroy', 'id': 4077, 'Fra...","[{'Naam': 'Celia Groothedde', 'id': 4398, 'Fra...","Commissie voor Algemeen Beleid, Financiën, Beg...",2023-10-17,"[{'Naam': 'Adeline Blancquaert', 'id': 4386, '...","[{'Naam': 'Andries Gryffroy', 'id': 4077, 'Fra...","[{'Naam': 'Johan Deckmyn', 'id': 2838, 'Fracti...",13.0,2.0,5.0,10.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vergadering 1781516,"[{'Naam': 'Filip Brusselmans', 'id': 4387, 'Fr...","[{'Naam': 'Katrien Partyka', 'id': 4059, 'Frac...","[{'Naam': 'Sihame El Kaouakibi', 'id': 4396, '...",Bijzondere commissie voor de evaluatie van de ...,2023-12-01,"[{'Naam': 'Filip Brusselmans', 'id': 4387, 'Fr...","[{'Naam': 'Katrien Partyka', 'id': 4059, 'Frac...","[{'Naam': 'Sihame El Kaouakibi', 'id': 4396, '...",11.0,3.0,4.0,10.0,3.0,4.0
Vergadering 1784136,"[{'Naam': 'Ann De Martelaer', 'id': 1781, 'Fra...","[{'Naam': 'Filip Brusselmans', 'id': 4387, 'Fr...","[{'Naam': 'Suzy Wouters', 'id': 4429, 'Fractie...",Bijzondere commissie voor de evaluatie van de ...,2023-12-08,"[{'Naam': 'Koen Daniëls', 'id': 4070, 'Fractie...","[{'Naam': 'Filip Brusselmans', 'id': 4387, 'Fr...","[{'Naam': 'Suzy Wouters', 'id': 4429, 'Fractie...",10.0,6.0,1.0,8.0,6.0,1.0
Vergadering 1784175,"[{'Naam': 'Jean-Jacques De Gucht', 'id': 3416,...","[{'Naam': 'Filip Brusselmans', 'id': 4387, 'Fr...","[{'Naam': 'Suzy Wouters', 'id': 4429, 'Fractie...",Bijzondere commissie voor de evaluatie van de ...,2023-12-08,"[{'Naam': 'Koen Daniëls', 'id': 4070, 'Fractie...","[{'Naam': 'Filip Brusselmans', 'id': 4387, 'Fr...","[{'Naam': 'Suzy Wouters', 'id': 4429, 'Fractie...",9.0,6.0,1.0,8.0,6.0,1.0
Vergadering 1763400,"[{'Naam': 'Cathy Coudyser', 'id': 4068, 'Fract...","[{'Naam': 'Stephanie D'Hose', 'id': 4395, 'Fra...","[{'Naam': 'Celia Groothedde', 'id': 4398, 'Fra...",Interparlementaire Commissie van de Nederlands...,2023-10-02,"[{'Naam': 'Cathy Coudyser', 'id': 4068, 'Fract...","[{'Na

In [15]:
# Define function to obtain counters for the relevant dataframes
def obtain_attendance_counter(DataFrame_column_attendance):
    # Create empty list to store all names for relevant attendance status
    flattened_list_attendance = []
    
    # Iterate over all meetings and the corresponding lists for this attendance status
    for attendance_list in DataFrame_column_attendance:
        # Check if the attendance_list is NaN or not a list
        if attendance_list is not np.nan and isinstance(attendance_list, list):
            # Extract only names and extend to list (to ensure flat list)
            attendance_names = [item["Naam"] for item in attendance_list]
            flattened_list_attendance.extend(attendance_names)
     
    # Count occurrences of members for all meetings represented in attandance_list
    element_counts = Counter(flattened_list_attendance)
    
    # Sort elements by their counts in descending order
    sorted_counts = sorted(element_counts.items(), key=lambda x: x[1], reverse=True)

    return sorted_counts

In [16]:
# Obtain average presence for all meetings of each commission (for both alle and vaste leden)
# Extracting the last 6 columns
relevant_counts_attendance = meetings_all_commissions_filtered_df.iloc[:, -6:]
# Calculate average for each column
average_relevant_counts_attendance = relevant_counts_attendance.mean()

In [17]:
# Amend commissions_overview_filtered_df with overview how often each member was present / absent / verontschuldigd
# + include averages attendances over all meetings of each commission

# Iterate over each commission
for index_overview, row_overview in commissions_overview_filtered_df.iterrows():
    # only modify dataframe with respect to commissions for which meetings were held
    if row_overview["aantal vergaderingen"] != 0:
        # Obtain commission title and its permanent members
        spec_commission_title = row_overview["commissie.titel"]

        # Select relevant meetings for this commission out of meetings_all_commissions_filtered_df
        # If only 1 meeting was selected, this results in the same dataframe
        meetings_all_commissions_filtered_df_relevant = meetings_all_commissions_filtered_df[
            meetings_all_commissions_filtered_df["commissie.titel"] == spec_commission_title]



        # Obtain how often (permanent) members were present, absent and absent with notice
        for column_name_overview, column_name_spec in zip(['aanwezig_count_alle', 'afwezig_count_alle', 'verontschuldigd_count_alle',
                                                           'aanwezig_count_vaste', 'afwezig_count_vaste', 'verontschuldigd_count_vaste'],
                                                          ['AANWEZIG', 'AFWEZIG', 'VERONTSCHULDIGD',
                                                           'AANWEZIG_vast', 'AFWEZIG_vast', 'VERONTSCHULDIGD_vast']):
            commissions_overview_filtered_df.at[index_overview, column_name_overview] = obtain_attendance_counter(
                meetings_all_commissions_filtered_df_relevant[column_name_spec])

        # Obtain average presence for all meetings of this commission (for both alle and vaste leden)
        # Extracting the last 6 columns, i.e. where attendance counts are stored
        relevant_counts_attendance = meetings_all_commissions_filtered_df_relevant.iloc[:, -6:]
        # Calculate average for each column
        average_relevant_counts_attendance = relevant_counts_attendance.mean()

        # Iterate over the column names and assign the values to the DataFrame
        for i, col_name_overview in enumerate(['Gemiddelde aantal aanwezig alle leden', 'Gemiddelde aantal afwezig alle leden',
                                      'Gemiddelde aantal verontschuldigd alle leden',
                                      'Gemiddelde aantal aanwezig vaste leden', 'Gemiddelde aantal afwezig vaste leden',
                                      'Gemiddelde aantal verontschuldigd vaste leden']):
            commissions_overview_filtered_df.loc[index_overview, col_name_overview] = average_relevant_counts_attendance.iloc[i]
    

In [18]:
# Inspect results
commissions_overview_filtered_df

,commissie.id,commissie.titel,commissie.link,voorzitter,eerste ondervoorzitter,tweede ondervoorzitter,derde ondervoorzitter,vierde ondervoorzitter,vaste leden,plaatsvervangende leden,...,verontschuldigd_count_alle,aanwezig_count_vaste,afwezig_count_vaste,verontschuldigd_count_vaste,Gemiddelde aantal aanwezig alle leden,Gemiddelde aantal afwezig alle leden,Gemiddelde aantal verontschuldigd alle leden,Gemiddelde aantal aanwezig vaste leden,Gemiddelde aantal afwezig vaste leden,Gemiddelde aantal verontschuldigd vaste leden
0,1622680,Commissie Vlaams Energie- en Klimaatplan,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Andries Gryffroy],[Bruno Tobback],[Robrecht Bothuyne],[],None,"[Andries Gryffroy, Bruno Tobback, Robrecht Bot...","[Adeline Blancquaert, Bart Claes, Inez De Coni...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
1,1332937,"Commissie voor Algemeen Beleid, Financiën, Beg...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Steven Vandeput],[Willem-Frederik Schiltz],[Arnout Coel],[],None,"[Steven Vandeput, Willem-Frederik Schiltz, Arn...","[Filip Brusselmans, Yves Buysse, Stijn De Roo,...",...,"[(Andries Gryffroy, 9), (Celia Groothedde, 8),...","[(Steven Vandeput, 12), (Arnout Coel, 11), (Bj...","[(Adeline Blancquaert, 7), (Peter Van Rompuy, ...","[(Andries Gryffroy, 9), (Celia Groothedde, 8),...",11.583333,2.583333,2.916667,8.750000,2.583333,2.416667
2,1332966,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Kris Van Dijck],[Sam Van Rooy],[Bart Tommelein],[],None,"[Kris Van Dijck, Sam Van Rooy, Bart Tommelein,...","[Adeline Blancquaert, Guy D'haeseleer, Stephan...",...,"[(Elke Sleurs, 7), (Tom Ongena, 6), (Celia Gro...","[(Nadia Sminate, 11), (Kris Van Dijck, 11), (M...","[(An Moerenhout, 9), (Maxim Veys, 8), (Chris J...","[(Paul Van Miert, 4), (An Moerenhout, 3), (Maa...",12.333333,3.166667,3.666667,8.500000,3.166667,1.416667
3,1332996,Commissie voor Brussel en de Vlaamse Rand en D...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Mercedes Van Volcem],[Karl Vanlouwe],[Klaas Slootmans],[],None,"[Mercedes Van Volcem, Karl Vanlouwe, Klaas Slo...","[Meyrem Almaci, Karin Brouwers, Filip Brusselm...",...,"[(Bart Dochy, 3), (Katia Segers, 2), (Gwenny D...","[(Jan Laeremans, 6), (Allessia Claes, 5), (Sof...","[(An Moerenhout, 5), (Stijn Bex, 3), (Peter Va...","[(Bart Dochy, 3), (Katia Segers, 2), (Gwenny D...",12.833333,3.500000,2.833333,9.166667,3.500000,2.333333
4,1333030,"Commissie voor Buitenlands Beleid, Europese Aa...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Cathy Coudyser],[Johan Deckmyn],[Tine Van den Brande],[],None,"[Cathy Coudyser, Johan Deckmyn, Tine Van den B...","[Adeline Blancquaert, Jean-Jacques De Gucht, M...",...,"[(Philippe Muyters, 7), (Joris Nachtergaele, 7...","[(Cathy Coudyser, 10), (Johan Deckmyn, 10), (S...","[(Carmen Ryheul, 8), (Tine Van den Brande, 6),...","[(Philippe Muyters, 7), (Joris Nachtergaele, 7...",10.300000,2.900000,4.800000,8.000000,2.900000,4.400000
5,1333053,"Commissie voor Cultuur, Jeugd, Sport en Media",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Meyrem Almaci],[Marius Meremans],[Karin Brouwers],[],None,"[Meyrem Almaci, Marius Meremans, Karin Brouwer...","[Immanuel De Reuse, Maaike De Rudder, Andries ...",...,"[(Karin Brouwers, 7), (Tine Van den Brande, 7)...","[(Meyrem Almaci, 15), (Freya Perdaens, 14), (M...","[(Wilfried Vandaele, 14), (Jeremie Vaneeckhout...","[(Karin Brouwers, 7), (Steven Coenegrachts, 6)...",10.000000,5.722222,3.777778,6.777778,5.722222,2.555556
6,1333088,"Commissie voor Economie, Werk, Sociale Economi...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Robrecht Bothuyne],[Imade Annouri],[Andries Gryffroy],[],None,"[Robrecht Bothuyne, Imade Annouri, Andries Gry...","[Steven Coenegrachts, Johan Danen, Annick De R...",...,"[(Rita Moors, 12), (Maaike De Vreese, 9), (Tom...","[(Robrecht Bothuyne, 16), (Thijs Verbeurgt, 14...","[(Meyrem Almaci, 16), (Yves Buysse, 11), (Imad...","[(Rita Moors, 12

# Analysis results

## Gemiddelde aanwezigheid vaste leden in commissies

In [ ]:
# ############################# Dash code #############################
# # Update function for pie charts
# def update_pie_charts(selected_data):    
#     fig = go.Figure()

#     for index, row in selected_data.iterrows():
#         data_present = row['Gemiddelde aantal aanwezig vaste leden']
#         data_absent = row['Gemiddelde aantal afwezig vaste leden']
#         data_excused = row['Gemiddelde aantal verontschuldigd vaste leden']

#         if not pd.isnull(data_present) and not pd.isnull(data_absent) and not pd.isnull(data_excused):
#             labels = ['Aanwezig', 'Afwezig', 'Verontschuldigd']
#             sizes = [data_present, data_absent, data_excused]
#             colors = ['green', 'red', 'orange']

#             # Create a pie chart trace
#             pie_chart = go.Pie(
#                 labels=labels,
#                 values=sizes,
#                 textinfo='percent',
#                 # hovertemplate="aantal leden: %{value}", # see https://plotly.com/python/hover-text-and-formatting/
#                 hoverinfo='value',
#                 hole=0.3,
#                 marker=dict(colors=colors),
#                 name=row['commissie.titel']
#             )
#             fig.add_trace(pie_chart)
#         else:
#             fig.add_trace(go.Scatter(
#                 x=[],
#                 y=[],
#                 mode='text',
#                 text=[f"Onvoldoende data beschikbaar voor {row['commissie.titel']}"],
#                 name=row['commissie.titel'],
#                 textposition='top center',
#                 showlegend=False
#             ))

#     fig.update_layout(
#         title='Gemiddelde aanwezigheid van vaste leden per commissie',
#         grid={'rows': 1, 'columns': 1},
#         showlegend=True
#     )

#     return fig


In [ ]:
# ############################ pythond code #############################
# # Plotting pie charts for each row in the DataFrame
# for index, row in commissions_overview_df.iterrows():
#     # When no attendance has been registered, no list is available. Address with if-else
#     if isinstance(row['Gemiddelde aantal aanwezig / afwezig kennisgeving / afwezig vaste leden (afgerond)'], list):
#         labels = ['Aanwezig', 'Afwezig', 'Verontschuldigd']
#         sizes = row['Gemiddelde aantal aanwezig / afwezig kennisgeving / afwezig vaste leden (afgerond)']
#         colors = ['green', 'red', 'orange']
#         # explode = (0.1, 0, 0)  # Explode the 1st slice (Present)
#         # explode = (0.1, 0.1, 0.1)  # Explode all slices
#         explode = None

#         plt.figure(figsize=(6, 6))
#         _, _, autotexts = plt.pie(sizes, 
#                                   # labels=labels, 
#                                   colors=colors,
#                               autopct='', startangle=140, explode=explode)

#         for i, autotext in enumerate(autotexts):
#             # autotext.set_text(str(sizes[i]))  # Set the original sizes as labels
#             autotext.set_text(str(int(round(sizes[i]))))  # Round and cast to integer
#             # autotext.set_text(str(round(sizes[i], 1)))  # Round 

#         plt.title(row['commissie.titel'], loc='center')
#         plt.axis('equal')

#         # Place the legend below the graph
#         plt.legend(labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), shadow=True, ncol=len(labels))
        
#         # Save the plot as a PNG file
#         plt.savefig(f'../graphs/average_presence_permanent_members/{today_str}_attendance_per_party_{row["commissie.titel"]}.png', 
#                     dpi=300, bbox_inches='tight')
        
#         plt.show()
#     else:
#         print("-" * 100)
#         print(f"No data available for {row['commissie.titel']}.")
#         print("-" * 100)

## Aanwezigheid van parlementsleden die vast commissielid zijn

### In hoeveel commissies zit elk parlementslid als vast lid? 

In [19]:
def count_member_occurrence(attendance_list: list):
    '''
    Count how many members were present / absent / absent with notice for each of the relevant meetings.
    '''
    # Flatten the list of lists to count attendance over all meetings in the attendance list
    # Skip empty cells that seemed to appear, as this is not informative.
    flattened_attendance = [item for sublist in attendance_list for item in sublist if item != '']

    # Count occurrences of members for all meetings represented in attandance_list
    element_counts = Counter(flattened_attendance)
    
    # Sort elements by their counts in descending order
    sorted_counts = sorted(element_counts.items(), key=lambda x: x[1], reverse=True)

    return sorted_counts

In [20]:
def amount_commissions_as_permanent_dict(commissions_overview_df_input, 
                                    parlementsleden_all_dict_input, 
                                    fracties_dict_input):
    """
    Function to obtain an overview of how many commissions each member of parliament is a permanent member of.
    """
    
     # Count how often a member occurs in the overview of lists of permanent members of commissions
    amount_commissions_members = count_member_occurrence(commissions_overview_df_input['vaste leden'])
    
    
    # Group members per amount of commissions
    grouped_by_count = defaultdict(list)
    # Grouping elements by their count
    for name, count in amount_commissions_members:
        grouped_by_count[count].append(name)


    # Add members that are in no commissions at all (with count = 0)
    # Obtain list of all member that are in at least 1 commission as permanent member
    permanent_members = [name for names in grouped_by_count.values() for name in names]
    # Obtain list of all members of parliament (i.e. including those not in any commission)
    parlementsleden_list = [parlementsleden_all_dict[member_tuple][0] for member_tuple in parlementsleden_all_dict_input.keys()]
    # Obtain list of all members of parliament that do not reside in any commission as permanent member
    not_permanent_members = list(set(parlementsleden_list).difference(set(permanent_members)))
    # Add members that are in no commission as permanent member to dict
    grouped_by_count[0] = not_permanent_members
    
    name2amount_dict = {}
    for number, names in grouped_by_count.items():
        for name in names:
            name2amount_dict[name] = number
    
    return name2amount_dict

In [21]:
# apply to current data
name2count_permanent_dict = amount_commissions_as_permanent_dict(
                    commissions_overview_df_input = commissions_overview_df,
                    parlementsleden_all_dict_input = parlementsleden_all_dict, 
                    fracties_dict_input = fracties_dict)

# Inspect results
name2count_permanent_dict

{'Willem-Frederik Schiltz': 12,
 'Chris Janssens': 10,
 'Peter Van Rompuy': 9,
 "Jos D'Haese": 9,
 'Andries Gryffroy': 7,
 'Bart Tommelein': 7,
 'Joke Schauvliege': 7,
 'Stefaan Sintobin': 7,
 'Allessia Claes': 6,
 'Kristof Slagmulder': 6,
 'Imade Annouri': 6,
 'Koen Daniëls': 6,
 'Arnout Coel': 5,
 'Björn Rzoska': 5,
 'Annabel Tavernier': 5,
 'Freya Perdaens': 5,
 'Staf Aerts': 4,
 'Rita Moors': 4,
 'Philippe Muyters': 4,
 'Sam Van Rooy': 4,
 'Maaike De Vreese': 4,
 'Celia Groothedde': 4,
 'Yves Buysse': 4,
 'An Moerenhout': 4,
 'Nadia Sminate': 4,
 'Karl Vanlouwe': 4,
 'Gwenny De Vroe': 4,
 'Sofie Joosen': 4,
 'Jan Laeremans': 4,
 'Cathy Coudyser': 4,
 'Joris Nachtergaele': 4,
 'Manuela Van Werde': 4,
 'Wilfried Vandaele': 4,
 'Jeremie Vaneeckhout': 4,
 'Axel Ronse': 4,
 'Kathleen Krekels': 4,
 'Katja Verheyen': 4,
 'Kurt De Loor': 4,
 'Bruno Tobback': 3,
 'Steven Coenegrachts': 3,
 'Leo Pieters': 3,
 'Wim Verheyden': 3,
 'Maurits Vande Reyde': 3,
 'Katrien Partyka': 3,
 'Stijn Bex':

### Welke parlementsleden zetelen in geen enkele commissie als vast lid? 

In [22]:
# Obtain list of all members of parliament
parlementsleden_list = [parlementsleden_all_dict[member_tuple][0] for member_tuple in parlementsleden_all_dict.keys()]
# print(parlementsleden_list)

# Obtain list of all members of parliament that reside in commissions as permanent member
all_permanent_members = set()
for member_list in commissions_overview_df['vaste leden']:
    all_permanent_members.update(member_list)

# Obtain list of all members of parliament that do not reside in any commission as permanent member
not_permanent_member = [element for element in parlementsleden_list if element not in list(all_permanent_members)]
print("Parlementleden die in geen enkele commissie zetelen als vast lid:", not_permanent_member)

Parlementleden die in geen enkele commissie zetelen als vast lid: ['Els Ampe', 'Bart De Wever', 'Bart Somers']


### Wie was het meest aanwezig bij de commissies waarin deze als vast lid zetelen? 

We obtain aggregated counts per member over all commissions, to get overall totals per member how often they had a certain attendance status (e.g. 'Aanwezig', 'Afwezig', 'Verontschuldigd'), using a helper function `obtain_aggregated_counts()`.

In [23]:
def obtain_aggregated_counts(dataframe_column):
    """
    Create function to aggregate counts per member over all commission, to get overall totals per member
    """
    # Create a defaultdict to store aggregated counts per member
    aggregated_counts = defaultdict(int)

    # Iterate through each row and aggregate counts per member
    for row in dataframe_column:
        for member, count in row:
            aggregated_counts[member] += count

    # Convert aggregated counts dictionary to a DataFrame
    aggregated_df = pd.DataFrame(list(aggregated_counts.items()), columns=['Member', 'Aggregated_Count'])

    aggregated_df

    # Create a defaultdict to store aggregated counts per member
    aggregated_counts = defaultdict(int)

    # Iterate through each row and aggregate counts per member
    for row in dataframe_column:
        for member, count in row:
            aggregated_counts[member] += count

    # Convert aggregated counts dictionary to a DataFrame
    aggregated_df = pd.DataFrame(list(aggregated_counts.items()), columns=['Member', 'Aggregated_Count'])
    
    # Sort the DataFrame based on Aggregated_Count in descending order
    aggregated_df = aggregated_df.sort_values(by='Aggregated_Count', ascending=False)
  
    return aggregated_df

In [24]:
def find_member(dictionary, member_name):
    """
    Function to add member of party to dataframe
    """
    for key, value in dictionary.items():
        for member in value:
            if member[0] == member_name:
                return key
    return None  # Return None if member is not found in any group

In [42]:
commissions_overview_df.head()

,commissie.id,commissie.titel,commissie.link,voorzitter,eerste ondervoorzitter,tweede ondervoorzitter,derde ondervoorzitter,vierde ondervoorzitter,vaste leden,plaatsvervangende leden,...,aanwezig_count_vaste,afwezig_count_vaste,verontschuldigd_count_vaste,Gemiddelde aantal aanwezig alle leden,Gemiddelde aantal afwezig alle leden,Gemiddelde aantal verontschuldigd alle leden,Gemiddelde aantal aanwezig vaste leden,Gemiddelde aantal afwezig vaste leden,Gemiddelde aantal verontschuldigd vaste leden,aanwezig_count_alle_maar_geen_vast_lid
0,1622680,Commissie Vlaams Energie- en Klimaatplan,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Andries Gryffroy],[Bruno Tobback],[Robrecht Bothuyne],[],None,"[Andries Gryffroy, Bruno Tobback, Robrecht Bot...","[Adeline Blancquaert, Bart Claes, Inez De Coni...",...,,,,NaN,NaN,NaN,NaN,NaN,NaN,[]
1,1332937,"Commissie voor Algemeen Beleid, Financiën, Beg...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Steven Vandeput],[Willem-Frederik Schiltz],[Arnout Coel],[],None,"[Steven Vandeput, Willem-Frederik Schiltz, Arn...","[Filip Brusselmans, Yves Buysse, Stijn De Roo,...",...,"[(Steven Vandeput, 12), (Arnout Coel, 11), (Bj...","[(Adeline Blancquaert, 7), (Peter Van Rompuy, ...","[(Andries Gryffroy, 9), (Celia Groothedde, 8),...",11.583333,2.583333,2.916667,8.750000,2.583333,2.416667,"[(Thijs Verbeurgt, 6), (Jeroen Tiebout, 6), (K..."
2,1332966,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Kris Van Dijck],[Sam Van Rooy],[Bart Tommelein],[],None,"[Kris Van Dijck, Sam Van Rooy, Bart Tommelein,...","[Adeline Blancquaert, Guy D'haeseleer, Stephan...",...,"[(Nadia Sminate, 11), (Kris Van Dijck, 11), (M...","[(An Moerenhout, 9), (Maxim Veys, 8), (Chris J...","[(Paul Van Miert, 4), (An Moerenhout, 3), (Maa...",12.333333,3.166667,3.666667,8.500000,3.166667,1.416667,"[(Kurt De Loor, 8), (Koen Van den Heuvel, 8), ..."
3,1332996,Commissie voor Brussel en de Vlaamse Rand en D...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Mercedes Van Volcem],[Karl Vanlouwe],[Klaas Slootmans],[],None,"[Mercedes Van Volcem, Karl Vanlouwe, Klaas Slo...","[Meyrem Almaci, Karin Brouwers, Filip Brusselm...",...,"[(Jan Laeremans, 6), (Allessia Claes, 5), (Sof...","[(An Moerenhout, 5), (Stijn Bex, 3), (Peter Va...","[(Bart Dochy, 3), (Katia Segers, 2), (Gwenny D...",12.833333,3.500000,2.833333,9.166667,3.500000,2.333333,"[(Ludwig Vandenhove, 4), (Meyrem Almaci, 3), (..."
4,1333030,"Commissie voor Buitenlands Beleid, Europese Aa...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Cathy Coudyser],[Johan Deckmyn],[Tine Van den Brande],[],None,"[Cathy Coudyser, Johan Deckmyn, Tine Van den B...","[Adeline Blancquaert, Jean-Jacques De Gucht, M...",...,"[(Cathy Coudyser, 10), (Johan Deckmyn, 10), (S...","[(Carmen Ryheul, 8), (Tine Van den Brande, 6),...","[(Philippe Muyters, 7), (Joris Nachtergaele, 7...",10.300000,2.900000,4.800000,8.000000,2.900000,4.400000,"[(Sofie Joosen, 7), (Orry Van de Wauwer, 4), (..."


In [81]:

def obtain_attendance_permanent_members(commissions_overview_df_input, fracties_dict_input, name2count_permanent_dict_input):
    """
    We obtain aggregated counts per member over all commissions, to get overall totals per member how often they had a certain attendance status (e.g. 'Aanwezig', 'Afwezig', 'Verontschuldigd'), using a helper function `obtain_aggregated_counts()`.
    """
    # Obtaining actual aggregated counts for how often permanent members were present / absent / absent with notice
    aanwezig_per_lid = obtain_aggregated_counts(commissions_overview_df_input['aanwezig_count_vaste']).rename(columns={"Aggregated_Count": "Aantal vergaderingen aanwezig"})
    afezig_per_lid = obtain_aggregated_counts(commissions_overview_df_input['afwezig_count_vaste']).rename(columns={"Aggregated_Count": "Aantal vergaderingen afwezig"})
    verontschuldigd_per_lid = obtain_aggregated_counts(commissions_overview_df_input['verontschuldigd_count_vaste']).rename(columns={"Aggregated_Count": "Aantal vergaderingen verontschuldigd"})
    #Merge those dfs, using 'Member' column as identifier
    merged_df = pd.merge(aanwezig_per_lid, afezig_per_lid,  on="Member", how="outer")
    merged_df = pd.merge(merged_df, verontschuldigd_per_lid,  on="Member", how="outer") 
    # Set member names as index (to allow easier processing later on)
    aanwezigheid_per_lid_df = merged_df.set_index("Member")

    # There seems to be some members present in `all_permanent_members` (which is based on `commissions_overview_df['vaste leden']`) that are not present in `aanwezig_per_lid` (which is based on `commissions_overview_df['aanwezig_count_vaste']`). We assess which these are, and add them to the `aanwezig_per_lid` dataframe, with count 0.
    # Obtain list of all members of parliament that reside in commissions as permanent member
    all_permanent_members = set()
    for member_list in commissions_overview_df_input['vaste leden']:
        all_permanent_members.update(member_list)
    # Obtain list of all members of parliament that do not reside in any commission as permanent member
    permanent_members_never_present = [element for element in all_permanent_members if element not in aanwezigheid_per_lid_df.index]
    # Add those members to the dataframe, with counts of 0
    for member in permanent_members_never_present:
        aanwezigheid_per_lid_df.loc[member] = 0
        
    #Add extra columns for amount of relevant meetings, percentage and amount commissions as permanent member
    aanwezigheid_per_lid_df['Aantal relevante vergaderingen'] = 0
    aanwezigheid_per_lid_df['Percentage vergaderingen aanwezig'] = 0.0 # initialise as float
    aanwezigheid_per_lid_df['Aantal commissies waarin vast lid'] = 0
    #Fill amount of relevant meetings: Iterate over all permanent members of parliament
    for index_members, row_members in aanwezigheid_per_lid_df.iterrows():
        # Iterate for each member through all commissions. 
        # If they are a permanent member, add the amount of meetings of that commission to their key in the dict
        for index_overview, row_overview in commissions_overview_df.iterrows():
            if index_members in row_overview["vaste leden"]:
                aanwezigheid_per_lid_df.at[index_members, 'Aantal relevante vergaderingen'] += row_overview['aantal vergaderingen']
    
    print(aanwezigheid_per_lid_df.columns)
                
    
    #Iterate over all permanent members of parliament
    for index_members, row_members in aanwezigheid_per_lid_df.iterrows():
        # Obtain percentage of attended meetings (taking zero division into account, i.e. with Aantal relevante vergaderingen == 0)
        percentage_attendance = row_members['Aantal vergaderingen aanwezig'] / row_members['Aantal relevante vergaderingen'] if row_members['Aantal relevante vergaderingen'] != 0 else 0.0      
        aanwezigheid_per_lid_df.at[index_members, "Percentage vergaderingen aanwezig"] = percentage_attendance

    # Sort dataframe by percentage attended
    aanwezigheid_per_lid_df = aanwezigheid_per_lid_df.sort_values(by = "Percentage vergaderingen aanwezig", ascending = False)
    
    # Add parties of relevant members
    parties_list = []
    for member in aanwezigheid_per_lid_df.index:
        party = find_member(fracties_dict_input, member)
        parties_list.append(party)       
    aanwezigheid_per_lid_df["Partij"] = parties_list
    
    # Mapping members to the amount of commission they are permanent member of
    # using the grouped_by_count_dict obtained before
    aanwezigheid_per_lid_df['Aantal commissies waarin vast lid'] = aanwezigheid_per_lid_df.index.map(name2count_permanent_dict_input)
    
    print(aanwezigheid_per_lid_df.columns)
    return aanwezigheid_per_lid_df



    

In [82]:
obtain_attendance_permanent_members(commissions_overview_df, 
                                    fracties_dict, 
                                    name2count_permanent_dict)

Index(['Aantal vergaderingen aanwezig', 'Aantal vergaderingen afwezig',
       'Aantal vergaderingen verontschuldigd',
       'Aantal relevante vergaderingen', 'Percentage vergaderingen aanwezig',
       'Aantal commissies waarin vast lid'],
      dtype='object')
Index(['Aantal vergaderingen aanwezig', 'Aantal vergaderingen afwezig',
       'Aantal vergaderingen verontschuldigd',
       'Aantal relevante vergaderingen', 'Percentage vergaderingen aanwezig',
       'Aantal commissies waarin vast lid', 'Partij'],
      dtype='object')


,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig,Aantal vergaderingen verontschuldigd,Aantal relevante vergaderingen,Percentage vergaderingen aanwezig,Aantal commissies waarin vast lid,Partij
Member,,,,,,,
Steven Vandeput,12.0,NaN,NaN,12,1.0,1,N-VA
Robrecht Bothuyne,16.0,NaN,NaN,16,1.0,2,cd&v
Jan Laeremans,36.0,NaN,NaN,36,1.0,4,Vlaams Belang
Liesbeth Homans,1.0,NaN,NaN,1,1.0,2,N-VA
Johan Danen,21.0,NaN,NaN,21,1.0,1,Groen
...,...,...,...,...,...,...,...
Jo Brouns,0.0,0.0,0.0,0,0.0,1,None
Tine Van den Brande,NaN,10.0,7.0,17,NaN,3,Groen
Frieda Verougstraete-Deschacht,NaN,4.0,5.0,9,NaN,1,Vlaams Belang


In [76]:
def obtain_attendance_non_permanent_members(commissions_overview_df_input, 
                                            parlementsleden_all_dict_input,
                                            fracties_dict_input):
    """
    We obtain aggregated counts per member over all commissions in which they
    were no permanent member, to get overall totals per member how often they
    had a certain attendance     status (e.g. 'Aanwezig', 'Afwezig', 'Verontschuldigd'), 
    using a helper function `obtain_aggregated_counts()`.
    """
    # Obtaining actual aggregated counts for how often members were present
    aanwezig_per_lid_alle = obtain_aggregated_counts(commissions_overview_df_input['aanwezig_count_alle'])
    # Set member names as index (to allow easier processing later on)
    aanwezig_per_lid_alle = aanwezig_per_lid_alle.set_index("Member")
    
    
    # Then we obtain a list of all members of parliament, to compare them with the attendance data.
    alle_parlementsleden_list = [parlementsleden_all_dict_input[member_tuple][0] for member_tuple in parlementsleden_all_dict_input.keys()]
    
    
    # Obtain list of all members of parliament that have never attended a commission
    all_members_never_present = [element for element in alle_parlementsleden_list if element not in aanwezig_per_lid_alle.index]

    
    # Then we obtain for each commission a list of members that have attended meetings of that commission although they are no permanent member of that commission.
    members_present_but_no_permanent_overall = [] # Create empty list to fill
    # Iterate over each commission
    for index_overview, row_overview in commissions_overview_df_input.iterrows():
        permanent_members = row_overview["vaste leden"] # Obtain permanent members of commission
        # Obtain list of all members that have been present at at least 1 meeting of this commission
        members_present_in_meetings = [member_tuple[0] for member_tuple in row_overview["aanwezig_count_alle"]]
        # Obtain subset of those members that are no permanent members
        members_present_but_no_permanent = [member for member in members_present_in_meetings if member not in permanent_members]
        # Append this subset to the overall list
        members_present_but_no_permanent_overall.append(members_present_but_no_permanent)
      
    
    # Then we obtain for each of those non-permanent members how often they were present at a specific commission.
    count_members_present_but_not_permanent_overall = []
    for index, spec_commission_attendance_list in enumerate(commissions_overview_df_input['aanwezig_count_alle']):
        count_members_present_but_not_permanent = [member_tuple for member_tuple in spec_commission_attendance_list if member_tuple[0] in members_present_but_no_permanent_overall[index]]
        count_members_present_but_not_permanent_overall.append(count_members_present_but_not_permanent)
    # Append resulting list as new column to overview data frame 
    commissions_overview_df_input["aanwezig_count_alle_maar_geen_vast_lid"] = count_members_present_but_not_permanent_overall
        
        
    # Obtaining actual aggregated counts for how often members were present
    aanwezig_per_lid_alle_maar_niet_vast_df = obtain_aggregated_counts(count_members_present_but_not_permanent_overall)
    # Use member name as index
    aanwezig_per_lid_alle_maar_niet_vast_df = aanwezig_per_lid_alle_maar_niet_vast_df.set_index('Member')

    
    # Then we add the party the dataframe, using the earlier defined helper function.
    parties_list = []
    for member in aanwezig_per_lid_alle_maar_niet_vast_df.index:
        party = find_member(fracties_dict_input, member)
        parties_list.append(party)
    aanwezig_per_lid_alle_maar_niet_vast_df["Partij"] = parties_list
    
    
    # We also add in how many additional commissions they were present.
    aanwezig_per_lid_alle_maar_niet_vast_df["Aantal commissies extra aanwezig"] = 0
    # Flatten list of attendance (including counts)
    flattened_attendance_list = [item for sublist in commissions_overview_df_input["aanwezig_count_alle_maar_geen_vast_lid"] for item in sublist]
    # Maintain only names and not counts
    flattened_attendance_list = [item[0] for item in flattened_attendance_list]
    # Count occurrences of members for all meetings represented in attandance_list
    amount_of_meetings_counter = Counter(flattened_attendance_list)
    # Sort elements by their counts in descending order
    amount_of_meetings_list = sorted(amount_of_meetings_counter.items(), key=lambda x: x[1], reverse=True)
    for member_name, count in amount_of_meetings_counter.items():
        aanwezig_per_lid_alle_maar_niet_vast_df.at[member_name, "Aantal commissies extra aanwezig"] = count
    
    
    # Finally we also add for each member in how many commissions they are a permanent member.
    aanwezig_per_lid_alle_maar_niet_vast_df["Aantal commissies waarin vast lid"] = 0 # Initialize new column
    # Create flattened list of all permanent member accross all commissions
    flattened_permanent_members = [item for sublist in commissions_overview_df_input['vaste leden'] for item in sublist]
    # Count occurrences of meetings for which member is permanent member
    amount_of_meetings_as_permanent_counter = Counter(flattened_permanent_members)
    #Store counts in column '"Aantal commissies waarin vast lid"'
    for member_name, count in amount_of_meetings_as_permanent_counter.items():
        if member_name in aanwezig_per_lid_alle_maar_niet_vast_df.index:
            aanwezig_per_lid_alle_maar_niet_vast_df.at[member_name, "Aantal commissies waarin vast lid"] = count


    # Rename column "Aggregated_Count" to "Aantal vergaderingen aanwezig"
    aanwezig_per_lid_alle_maar_niet_vast_df = aanwezig_per_lid_alle_maar_niet_vast_df.rename(columns={"Aggregated_Count": "Aantal vergaderingen aanwezig"})
    
    print(aanwezig_per_lid_alle_maar_niet_vast_df.columns)
    return aanwezig_per_lid_alle_maar_niet_vast_df

In [77]:
obtain_attendance_non_permanent_members(commissions_overview_df, 
                                            parlementsleden_all_dict,
                                            fracties_dict)

Index(['Aantal vergaderingen aanwezig', 'Partij',
       'Aantal commissies extra aanwezig',
       'Aantal commissies waarin vast lid'],
      dtype='object')


,Aantal vergaderingen aanwezig,Partij,Aantal commissies extra aanwezig,Aantal commissies waarin vast lid
Member,,,,
Thijs Verbeurgt,18,Vooruit,6,1
Stijn De Roo,18,cd&v,4,2
Jeroen Tiebout,16,N-VA,8,2
Steve Vandenberghe,15,Vooruit,1,1
Johan Danen,15,Groen,7,1
...,...,...,...,...
Els Sterckx,1,Vlaams Belang,1,2
Carmen Ryheul,1,Vlaams Belang,1,2
Suzy Wouters,1,Vlaams Belang,1,2


In [72]:
obtain_aggregated_counts(commissions_overview_df['aanwezig_count_vaste'])

,Member,Aggregated_Count
52,Freya Perdaens,36
28,Jan Laeremans,36
1,Arnout Coel,35
90,Kathleen Krekels,31
3,Maurits Vande Reyde,30
...,...,...
113,Filip Dewinter,3
107,Lise Vandecasteele,2
95,Roosmarijn Beckers,2
62,Onno Vandewalle,1


In [56]:
aanwezig_temp = obtain_aggregated_counts(commissions_overview_df['aanwezig_count_vaste']).rename(columns={"Aggregated_Count": "Aantal vergaderingen aanwezig"})
aanwezig_temp

afwezig_temp = obtain_aggregated_counts(commissions_overview_df['afwezig_count_vaste']).rename(columns={"Aggregated_Count": "Aantal vergaderingen afwezig"})
afwezig_temp

verontschuldigd_temp = obtain_aggregated_counts(commissions_overview_df['verontschuldigd_count_vaste']).rename(columns={"Aggregated_Count": "Aantal vergaderingen verontschuldigd"})
verontschuldigd_temp

,Member,Aantal vergaderingen aanwezig
52,Freya Perdaens,36
28,Jan Laeremans,36
1,Arnout Coel,35
90,Kathleen Krekels,31
3,Maurits Vande Reyde,30
...,...,...
113,Filip Dewinter,3
107,Lise Vandecasteele,2
95,Roosmarijn Beckers,2
62,Onno Vandewalle,1


,Member,Aantal vergaderingen afwezig
36,Wilfried Vandaele,21
45,Meyrem Almaci,19
37,Jeremie Vaneeckhout,18
14,Sam Van Rooy,18
2,Willem-Frederik Schiltz,17
...,...,...
49,Manuela Van Werde,1
48,Karin Brouwers,1
27,Bart Dochy,1
29,Mercedes Van Volcem,1


,Member,Aantal vergaderingen verontschuldigd
0,Andries Gryffroy,23
33,Kristof Slagmulder,19
71,Roosmarijn Beckers,19
46,Rita Moors,16
3,Maaike De Vreese,15
...,...,...
68,Martine Fournier,1
17,Katrien Partyka,1
16,Yves Buysse,1
34,Vera Jans,1


In [60]:
merged_df = pd.merge(aanwezig_temp, afwezig_temp,  on="Member", how="outer")
merged_df

merged_df = pd.merge(merged_df, verontschuldigd_temp,  on="Member", how="outer") 
merged_df

,Member,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig
0,Freya Perdaens,36.0,4.0
1,Jan Laeremans,36.0,NaN
2,Arnout Coel,35.0,3.0
3,Kathleen Krekels,31.0,NaN
4,Maurits Vande Reyde,30.0,9.0
...,...,...,...
112,Onno Vandewalle,1.0,NaN
113,Liesbeth Homans,1.0,NaN
114,Tine Van den Brande,NaN,10.0
115,Frieda Verougstraete-Deschacht,NaN,4.0


,Member,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig,Aantal vergaderingen verontschuldigd
0,Freya Perdaens,36.0,4.0,7.0
1,Jan Laeremans,36.0,NaN,NaN
2,Arnout Coel,35.0,3.0,4.0
3,Kathleen Krekels,31.0,NaN,3.0
4,Maurits Vande Reyde,30.0,9.0,1.0
...,...,...,...,...
113,Liesbeth Homans,1.0,NaN,NaN
114,Tine Van den Brande,NaN,10.0,7.0
115,Frieda Verougstraete-Deschacht,NaN,4.0,5.0
116,Annick De Ridder,NaN,2.0,8.0


In [62]:
aanwezig_per_lid = merged_df.set_index("Member")
aanwezig_per_lid

,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig,Aantal vergaderingen verontschuldigd
Member,,,
Freya Perdaens,36.0,4.0,7.0
Jan Laeremans,36.0,NaN,NaN
Arnout Coel,35.0,3.0,4.0
Kathleen Krekels,31.0,NaN,3.0
Maurits Vande Reyde,30.0,9.0,1.0
...,...,...,...
Liesbeth Homans,1.0,NaN,NaN
Tine Van den Brande,NaN,10.0,7.0
Frieda Verougstraete-Deschacht,NaN,4.0,5.0


In [63]:
# Obtaining actual aggregated counts for how often permanent members were present / absent / absent with notice
aanwezig_per_lid = obtain_aggregated_counts(commissions_overview_df['aanwezig_count_vaste'])
afezig_per_lid = obtain_aggregated_counts(commissions_overview_df['afwezig_count_vaste'])
verontschuldigd_per_lid = obtain_aggregated_counts(commissions_overview_df['verontschuldigd_count_vaste'])
#Merge those dfs, using 'Member' column as identifier
merged_df = pd.merge(aanwezig_temp, afwezig_temp,  on="Member", how="outer")
merged_df = pd.merge(merged_df, verontschuldigd_temp,  on="Member", how="outer") 
# Set member names as index (to allow easier processing later on)
aanwezig_per_lid = merged_df.set_index("Member")

aanwezig_per_lid

,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig,Aantal vergaderingen verontschuldigd
Member,,,
Freya Perdaens,36.0,4.0,7.0
Jan Laeremans,36.0,NaN,NaN
Arnout Coel,35.0,3.0,4.0
Kathleen Krekels,31.0,NaN,3.0
Maurits Vande Reyde,30.0,9.0,1.0
...,...,...,...
Liesbeth Homans,1.0,NaN,NaN
Tine Van den Brande,NaN,10.0,7.0
Frieda Verougstraete-Deschacht,NaN,4.0,5.0


In [64]:
# There seems to be some members present in `all_permanent_members` (which is based on `commissions_overview_df['vaste leden']`) that are not present in `aanwezig_per_lid` (which is based on `commissions_overview_df['aanwezig_count_vaste']`). We assess which these are, and add them to the `aanwezig_per_lid` dataframe, with count 0.
# Obtain list of all members of parliament that reside in commissions as permanent member
all_permanent_members = set()
for member_list in commissions_overview_df['vaste leden']:
    all_permanent_members.update(member_list)
# Obtain list of all members of parliament that do not reside in any commission as permanent member
permanent_members_never_present = [element for element in all_permanent_members if element not in aanwezig_per_lid.index]
# Add those members to the dataframe, with a count of 0
for member in permanent_members_never_present:
    aanwezig_per_lid.loc[member] = 0

In [65]:
aanwezig_per_lid

,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig,Aantal vergaderingen verontschuldigd
Member,,,
Freya Perdaens,36.0,4.0,7.0
Jan Laeremans,36.0,NaN,NaN
Arnout Coel,35.0,3.0,4.0
Kathleen Krekels,31.0,NaN,3.0
Maurits Vande Reyde,30.0,9.0,1.0
...,...,...,...
Sihame El Kaouakibi,NaN,NaN,5.0
Piet De Bruyn,0.0,0.0,0.0
Steve Vandenberghe,0.0,0.0,0.0


In [69]:
aanwezig_per_lid['Aantal relevante vergaderingen'] = 0
aanwezig_per_lid['Percentage vergaderingen aanwezig'] = 0.0
aanwezig_per_lid['Aantal commissies waarin vast lid'] = 0

aanwezig_per_lid


,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig,Aantal vergaderingen verontschuldigd,"(Aantal relevante vergaderingen, Percentage vergaderingen aanwezig, Aantal commissies waarin vast lid)",Aantal relevante vergaderingen,Percentage vergaderingen aanwezig,Aantal commissies waarin vast lid
Member,,,,,,,
Freya Perdaens,36.0,4.0,7.0,0,0,0.0,0
Jan Laeremans,36.0,NaN,NaN,0,0,0.0,0
Arnout Coel,35.0,3.0,4.0,0,0,0.0,0
Kathleen Krekels,31.0,NaN,3.0,0,0,0.0,0
Maurits Vande Reyde,30.0,9.0,1.0,0,0,0.0,0
...,...,...,...,...,...,...,...
Sihame El Kaouakibi,NaN,NaN,5.0,0,0,0.0,0
Piet De Bruyn,0.0,0.0,0.0,0,0,0.0,0
Steve Vandenberghe,0.0,0.0,0.0,0,0,0.0,0


In [70]:
#Iterate over all permanent members of parliament
for index_members, row_members in aanwezig_per_lid.iterrows():
    # Iterate for each member through all commissions. 
    # If they are a permanent member, add the amount of meetings of that commission to their key in the dict
    for index_overview, row_overview in commissions_overview_df.iterrows():
        if index_members in row_overview["vaste leden"]:
            aanwezig_per_lid.at[index_members, 'Aantal relevante vergaderingen'] += row_overview['aantal vergaderingen']
            
aanwezig_per_lid            

,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig,Aantal vergaderingen verontschuldigd,"(Aantal relevante vergaderingen, Percentage vergaderingen aanwezig, Aantal commissies waarin vast lid)",Aantal relevante vergaderingen,Percentage vergaderingen aanwezig,Aantal commissies waarin vast lid
Member,,,,,,,
Freya Perdaens,36.0,4.0,7.0,0,47,0.0,0
Jan Laeremans,36.0,NaN,NaN,0,36,0.0,0
Arnout Coel,35.0,3.0,4.0,0,42,0.0,0
Kathleen Krekels,31.0,NaN,3.0,0,34,0.0,0
Maurits Vande Reyde,30.0,9.0,1.0,0,40,0.0,0
...,...,...,...,...,...,...,...
Sihame El Kaouakibi,NaN,NaN,5.0,0,9,0.0,0
Piet De Bruyn,0.0,0.0,0.0,0,0,0.0,0
Steve Vandenberghe,0.0,0.0,0.0,0,0,0.0,0


In [71]:
for index_members, row_members in aanwezig_per_lid.iterrows():
    # Obtain percentage of attended meetings (taking zero division into account, i.e. with Aantal relevante vergaderingen == 0)
    percentage_attendance = row_members['Aantal vergaderingen aanwezig'] / row_members['Aantal relevante vergaderingen'] if row_members['Aantal relevante vergaderingen'] != 0 else 0.0      
    aanwezig_per_lid.at[index_members, "Percentage vergaderingen aanwezig"] = percentage_attendance
    
aanwezig_per_lid

,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig,Aantal vergaderingen verontschuldigd,"(Aantal relevante vergaderingen, Percentage vergaderingen aanwezig, Aantal commissies waarin vast lid)",Aantal relevante vergaderingen,Percentage vergaderingen aanwezig,Aantal commissies waarin vast lid
Member,,,,,,,
Freya Perdaens,36.0,4.0,7.0,0,47,0.765957,0
Jan Laeremans,36.0,NaN,NaN,0,36,1.000000,0
Arnout Coel,35.0,3.0,4.0,0,42,0.833333,0
Kathleen Krekels,31.0,NaN,3.0,0,34,0.911765,0
Maurits Vande Reyde,30.0,9.0,1.0,0,40,0.750000,0
...,...,...,...,...,...,...,...
Sihame El Kaouakibi,NaN,NaN,5.0,0,9,NaN,0
Piet De Bruyn,0.0,0.0,0.0,0,0,0.000000,0
Steve Vandenberghe,0.0,0.0,0.0,0,0,0.000000,0


In [ ]:
#Create empty dataframe with permanent members, fill with zero (i.e. starting count) and use parmanent members as index
member_amount_meetings_pd = pd.DataFrame(0, columns = ['Aantal relevante vergaderingen',
                                                       'Aantal vergaderingen aanwezig',
                                                       'Aantal vergaderingen afwezig',
                                                       'Aantal vergaderingen verontschuldigd',
                                                       'Percentage vergaderingen aanwezig',
                                                       'Aantal commissies waarin vast lid'],
                                        index = list(all_permanent_members))

In [25]:
def obtain_attendance_permanent_members(commissions_overview_df_input, fracties_dict_input, name2count_permanent_dict_input):
    """
    We obtain aggregated counts per member over all commissions, to get overall totals per member how often they had a certain attendance status (e.g. 'Aanwezig', 'Afwezig', 'Verontschuldigd'), using a helper function `obtain_aggregated_counts()`.
    """
    # Obtaining actual aggregated counts for how often permanent members were present
    aanwezig_per_lid = obtain_aggregated_counts(commissions_overview_df_input['aanwezig_count_vaste'])
    # Set member names as index (to allow easier processing later on)
    aanwezig_per_lid = aanwezig_per_lid.set_index("Member")

    # There seems to be some members present in `all_permanent_members` (which is based on `commissions_overview_df['vaste leden']`) that are not present in `aanwezig_per_lid` (which is based on `commissions_overview_df['aanwezig_count_vaste']`). We assess which these are, and add them to the `aanwezig_per_lid` dataframe, with count 0.
    # Obtain list of all members of parliament that reside in commissions as permanent member
    all_permanent_members = set()
    for member_list in commissions_overview_df_input['vaste leden']:
        all_permanent_members.update(member_list)
    # Obtain list of all members of parliament that do not reside in any commission as permanent member
    permanent_members_never_present = [element for element in all_permanent_members if element not in aanwezig_per_lid.index]
    # Add those members to the dataframe, with a count of 0
    for member in permanent_members_never_present:
        aanwezig_per_lid.loc[member] = 0
        
    #Create empty dataframe with permanent members, fill with zero (i.e. starting count) and use parmanent members as index
    member_amount_meetings_pd = pd.DataFrame(0, columns = ['Aantal relevante vergaderingen',
                                                           'Aantal vergaderingen aanwezig',
                                                           'Aantal vergaderingen afwezig',
                                                           'Aantal vergaderingen verontschuldigd',
                                                           'Percentage vergaderingen aanwezig',
                                                           'Aantal commissies waarin vast lid'],
                                            index = list(all_permanent_members))
    #Iterate over all permanent members of parliament
    for index_members, row_members in member_amount_meetings_pd.iterrows():
        # Iterate for each member through all commissions. 
        # If they are a permanent member, add the amount of meetings of that commission to their key in the dict
        for index_overview, row_overview in commissions_overview_df.iterrows():
            if index_members in row_overview["vaste leden"]:
                member_amount_meetings_pd.at[index_members, 'Aantal relevante vergaderingen'] += row_overview['aantal vergaderingen']
                
    
    #Iterate over all permanent members of parliament
    for index_members, row_members in member_amount_meetings_pd.iterrows():
        # Extract the aggregated count out of aanwezig_per_lid for the relevant member (i.e. serving as index) and add to member_amount_meetings_pd at relevant index
        verg_aanwezig = aanwezig_per_lid.loc[index_members, "Aggregated_Count"]
        member_amount_meetings_pd.at[index_members, "Aantal vergaderingen aanwezig"] = verg_aanwezig
        verg_afwezig = aanwezig_per_lid.loc[index_members, "Aggregated_Count"]
        member_amount_meetings_pd.at[index_members, "Aantal vergaderingen afwezig"] = verg_afwezig
        verg_verontschuldigd = aanwezig_per_lid.loc[index_members, "Aggregated_Count"]
        member_amount_meetings_pd.at[index_members, "Aantal vergaderingen verontschuldigd"] = verg_verontschuldigd
        
        # Obtain percentage of attended meetings (taking zero division into account, i.e. with Aantal relevante vergaderingen == 0)
        percentage_attendance = verg_aanwezig / row_members['Aantal relevante vergaderingen'] if row_members['Aantal relevante vergaderingen'] != 0 else 0      
        member_amount_meetings_pd.at[index_members, "Percentage vergaderingen aanwezig"] = percentage_attendance
    # Sort dataframe by percentage attended
    member_amount_meetings_pd = member_amount_meetings_pd.sort_values(by = "Percentage vergaderingen aanwezig", ascending = False)
    
    # Add parties of relevant members
    parties_list = []
    for member in member_amount_meetings_pd.index:
        party = find_member(fracties_dict_input, member)
        parties_list.append(party)       
    member_amount_meetings_pd["Partij"] = parties_list
    
    # Mapping members to the amount of commission they are permanent member of
    # using the grouped_by_count_dict obtained before
    member_amount_meetings_pd['Aantal commissies waarin vast lid'] = member_amount_meetings_pd.index.map(name2count_permanent_dict_input)
    
    
    return member_amount_meetings_pd


In [27]:
#Apply to relevant data
member_amount_meetings_df = obtain_attendance_permanent_members(
    commissions_overview_df,
    fracties_dict,
    name2count_permanent_dict)

# Inspect results
member_amount_meetings_df.head(20)

,Aantal relevante vergaderingen,Aantal vergaderingen aanwezig,Aantal vergaderingen afwezig,Aantal vergaderingen verontschuldigd,Percentage vergaderingen aanwezig,Aantal commissies waarin vast lid,Partij
Johan Danen,21,21,21,21,1.000000,1,Groen
Jan Laeremans,36,36,36,36,1.000000,4,Vlaams Belang
Steven Vandeput,12,12,12,12,1.000000,1,N-VA
Liesbeth Homans,1,1,1,1,1.000000,2,N-VA
Robrecht Bothuyne,16,16,16,16,1.000000,2,cd&v
Elke Sleurs,12,11,11,11,0.916667,2,N-VA
Kathleen Krekels,34,31,31,31,0.911765,4,N-VA
Katrien Schryvers,21,19,19,19,0.904762,2,cd&v
Loes Vandromme,21,19,19,19,0.904762,2,cd&v
Hilâl Yalçin,21,19,19,19,0.904762,1,cd&v


## Wie was het meest aanwezig bij de commissies waarin deze nochtans niet als vast lid zetelen? 

In [28]:
# # Load in data if necessary
# commissions_overview_df = pd.read_pickle('../data/overzicht_2023-12-06.pkl')

In [29]:
# Inspect relevant data parts
commissions_overview_df.columns
commissions_overview_df["aanwezig_count_alle"]

Index(['commissie.id', 'commissie.titel', 'commissie.link', 'voorzitter',
       'eerste ondervoorzitter', 'tweede ondervoorzitter',
       'derde ondervoorzitter', 'vierde ondervoorzitter', 'vaste leden',
       'plaatsvervangende leden', 'toegevoegde leden', 'aantal vergaderingen',
       'aanwezig_count_alle', 'afwezig_count_alle',
       'verontschuldigd_count_alle', 'aanwezig_count_vaste',
       'afwezig_count_vaste', 'verontschuldigd_count_vaste',
       'Gemiddelde aantal aanwezig alle leden',
       'Gemiddelde aantal afwezig alle leden',
       'Gemiddelde aantal verontschuldigd alle leden',
       'Gemiddelde aantal aanwezig vaste leden',
       'Gemiddelde aantal afwezig vaste leden',
       'Gemiddelde aantal verontschuldigd vaste leden'],
      dtype='object')

0                                                      
1     [(Steven Vandeput, 12), (Arnout Coel, 11), (Bj...
2     [(Nadia Sminate, 11), (Kris Van Dijck, 11), (M...
3     [(Jan Laeremans, 6), (Allessia Claes, 5), (Sof...
4     [(Cathy Coudyser, 10), (Johan Deckmyn, 10), (S...
5     [(Meyrem Almaci, 15), (Freya Perdaens, 14), (M...
6     [(Robrecht Bothuyne, 16), (Thijs Verbeurgt, 14...
7     [(Bart Dochy, 9), (Cathy Coudyser, 8), (Chris ...
8     [(Gwenny De Vroe, 18), (Inez De Coninck, 17), ...
9     [(Bart Claes, 9), (Marino Keulen, 8), (Sofie M...
10    [(Johan Danen, 21), (Jan Laeremans, 21), (Loes...
11    [(Stefaan Sintobin, 11), (Elke Sleurs, 11), (F...
12    [(Allessia Claes, 7), (Maxim Veys, 7), (Joke S...
13          [(Liesbeth Homans, 1), (Kris Van Dijck, 1)]
14    [(Chris Janssens, 2), (Axel Ronse, 2), (Willem...
15                                                     
16                                                     
17                                              

First, we obtain over overview of how often all members were present for the commission meetings, i.e. not limited to the permanent members as done earlier. We use the same helper function. 

In [30]:
# Obtaining actual aggregated counts for how often members were present
aanwezig_per_lid_alle = obtain_aggregated_counts(commissions_overview_df['aanwezig_count_alle'])

# Set member names as index (to allow easier processing later on)
aanwezig_per_lid_alle = aanwezig_per_lid_alle.set_index("Member")

# Inspect results
aanwezig_per_lid_alle

,Aggregated_Count
Member,
Freya Perdaens,49
Koen Daniëls,45
Arnout Coel,44
Kathleen Krekels,42
Jan Laeremans,38
...,...
Els Ampe,3
Roosmarijn Beckers,2
Lise Vandecasteele,2


Then we obtain a list of all members of parliament, to compare them with the attendance data.

In [31]:
alle_parlementsleden_list = [parlementsleden_all_dict[member_tuple][0] for member_tuple in parlementsleden_all_dict.keys()]
print(alle_parlementsleden_list)
len(alle_parlementsleden_list)

['Staf Aerts', 'Meyrem Almaci', 'Els Ampe', 'Hannes Anaf', 'Imade Annouri', 'Roosmarijn Beckers', 'Stijn Bex', 'Adeline Blancquaert', 'Robrecht Bothuyne', 'Karin Brouwers', 'Filip Brusselmans', 'Yves Buysse', 'Allessia Claes', 'Bart Claes', 'Arnout Coel', 'Steven Coenegrachts', 'Cathy Coudyser', 'Johan Danen', 'Koen Daniëls', 'Johan Deckmyn', 'Inez De Coninck', 'Jean-Jacques De Gucht', 'Kurt De Loor', 'Ann De Martelaer', 'Immanuel De Reuse', 'Annick De Ridder', 'Stijn De Roo', 'Maaike De Rudder', 'Frieda Verougstraete-Deschacht', 'Maarten De Veuster', 'Maaike De Vreese', 'Gwenny De Vroe', 'Bart De Wever', 'Filip Dewinter', 'Kim De Witte', "Jos D'Haese", "Guy D'haeseleer", "Stephanie D'Hose", 'Bart Dochy', 'Sihame El Kaouakibi', 'Martine Fournier', 'Veerle Geerinckx', 'Hannelore Goeman', 'Celia Groothedde', 'Karolien Grosemans', 'Andries Gryffroy', 'Liesbeth Homans', 'Vera Jans', 'Chris Janssens', 'Sofie Joosen', 'Marino Keulen', 'Kathleen Krekels', 'Jan Laeremans', 'Annick Lambrecht', 

123

In [32]:
# Obtain list of all members of parliament that have never attended a commission
all_members_never_present = [element for element in alle_parlementsleden_list if element not in aanwezig_per_lid_alle.index]
all_members_never_present

['Annick De Ridder',
 'Frieda Verougstraete-Deschacht',
 'Bart De Wever',
 'Kim De Witte',
 'Bart Somers',
 'Tine Van den Brande']

Then we obtain for each commission a list of members that have attended meetings of that commission although they are no permanent member of that commission.

In [33]:
members_present_but_no_permanent_overall = [] # Create empty list to fill

# Iterate over each commission
for index_overview, row_overview in commissions_overview_df.iterrows():
    permanent_members = row_overview["vaste leden"] # Obtain permanent members of commission
    # Obtain list of all members that have been present at at least 1 meeting of this commission
    members_present_in_meetings = [member_tuple[0] for member_tuple in row_overview["aanwezig_count_alle"]]
    # Obtain subset of those members that are no permanent members
    members_present_but_no_permanent = [member for member in members_present_in_meetings if member not in permanent_members]
    # Append this subset to the overall list
    members_present_but_no_permanent_overall.append(members_present_but_no_permanent)

members_present_but_no_permanent_overall

[[],
 ['Thijs Verbeurgt',
  'Jeroen Tiebout',
  'Katrien Schryvers',
  'Stijn De Roo',
  'Sarah Smeyers',
  'Freya Perdaens',
  'Johan Danen',
  'Hannelore Goeman',
  'Kurt De Loor',
  'Mieke Schauvliege',
  'Stefaan Sintobin',
  'Manuela Van Werde',
  'Kathleen Krekels',
  'Freya Saeys',
  'Jeremie Vaneeckhout',
  'Maxim Veys',
  'Inez De Coninck',
  'Veerle Geerinckx',
  'Bob Savenberg',
  'Nadia Sminate',
  'Annabel Tavernier',
  'Kris Van Dijck',
  'Paul Van Miert',
  'Brecht Warnez'],
 ['Kurt De Loor',
  'Koen Van den Heuvel',
  'Celia Groothedde',
  'Maurits Vande Reyde',
  'Orry Van de Wauwer',
  'Jeroen Tiebout',
  'Gwenny De Vroe',
  'Björn Rzoska',
  'Ludwig Vandenhove',
  'Kristof Slagmulder',
  'Imade Annouri',
  'Steven Coenegrachts',
  'Loes Vandromme',
  'Koen Daniëls',
  'Andries Gryffroy',
  'Stijn Bex',
  'Hannelore Goeman',
  "Stephanie D'Hose",
  'Johan Danen',
  'Ilse Malfroot',
  'Thijs Verbeurgt',
  'Tine van der Vloet',
  'Jean-Jacques De Gucht',
  'Adeline Blan

Then we obtain for each of those non-permanent members how often they were present at a specific commission.

In [35]:
count_members_present_but_not_permanent_overall = []
for index, spec_commission_attendance_list in enumerate(commissions_overview_df['aanwezig_count_alle']):
    count_members_present_but_not_permanent = [member_tuple for member_tuple in spec_commission_attendance_list if member_tuple[0] in members_present_but_no_permanent_overall[index]]
    count_members_present_but_not_permanent_overall.append(count_members_present_but_not_permanent)

# Append resulting list as new column to overview data frame 
commissions_overview_df["aanwezig_count_alle_maar_geen_vast_lid"] = count_members_present_but_not_permanent_overall

# Inspect data
commissions_overview_df["aanwezig_count_alle_maar_geen_vast_lid"]

0                                                    []
1     [(Thijs Verbeurgt, 6), (Jeroen Tiebout, 6), (K...
2     [(Kurt De Loor, 8), (Koen Van den Heuvel, 8), ...
3     [(Ludwig Vandenhove, 4), (Meyrem Almaci, 3), (...
4     [(Sofie Joosen, 7), (Orry Van de Wauwer, 4), (...
5     [(Katia Segers, 10), (Kathleen Krekels, 4), (M...
6     [(Björn Rzoska, 11), (Stijn De Roo, 10), (Elke...
7     [(Immanuel De Reuse, 1), (Tinne Rombouts, 1), ...
8     [(Robrecht Bothuyne, 13), (Willem-Frederik Sch...
9     [(Karin Brouwers, 5), (Stijn De Roo, 3), (Anni...
10    [(Steve Vandenberghe, 15), (Brecht Warnez, 7),...
11    [(Hannes Anaf, 8), (Vera Jans, 5), (Ilse Malfr...
12    [(Maxim Veys, 7), (Onno Vandewalle, 2), (Rita ...
13                                [(Kris Van Dijck, 1)]
14      [(Filip Dewinter, 1), (Sihame El Kaouakibi, 1)]
15                                                   []
16                                                   []
17                                              

In [36]:
# Obtaining actual aggregated counts for how often members were present
aanwezig_per_lid_alle_maar_niet_vast_df = obtain_aggregated_counts(commissions_overview_df['aanwezig_count_alle_maar_geen_vast_lid'])

# Use member name as index
aanwezig_per_lid_alle_maar_niet_vast_df = aanwezig_per_lid_alle_maar_niet_vast_df.set_index('Member')

# Inspect results
aanwezig_per_lid_alle_maar_niet_vast_df

,Aggregated_Count
Member,
Thijs Verbeurgt,18
Stijn De Roo,18
Jeroen Tiebout,16
Steve Vandenberghe,15
Johan Danen,15
...,...
Els Sterckx,1
Carmen Ryheul,1
Suzy Wouters,1


Then we add the party the dataframe, using the earlier defined helper function.

In [37]:
parties_list = []

for member in aanwezig_per_lid_alle_maar_niet_vast_df.index:
    party = find_member(fracties_dict, member)
    parties_list.append(party)
    
aanwezig_per_lid_alle_maar_niet_vast_df["Partij"] = parties_list

# Inspect results
aanwezig_per_lid_alle_maar_niet_vast_df

,Aggregated_Count,Partij
Member,,
Thijs Verbeurgt,18,Vooruit
Stijn De Roo,18,cd&v
Jeroen Tiebout,16,N-VA
Steve Vandenberghe,15,Vooruit
Johan Danen,15,Groen
...,...,...
Els Sterckx,1,Vlaams Belang
Carmen Ryheul,1,Vlaams Belang
Suzy Wouters,1,Vlaams Belang


We also add in how many additional commissions they were present.

In [ ]:
aanwezig_per_lid_alle_maar_niet_vast_df["Aantal commissies extra aanwezig"] = 0

# Inspect results
aanwezig_per_lid_alle_maar_niet_vast_df

In [ ]:
# Flatten list of attendance (including counts)
flattened_attendance_list = [item for sublist in commissions_overview_df["aanwezig_count_alle_maar_geen_vast_lid"] for item in sublist]
# Maintain only names and not counts
flattened_attendance_list = [item[0] for item in flattened_attendance_list]

# Inspect results
# print(flattened_attendance_list)

# Count occurrences of members for all meetings represented in attandance_list
amount_of_meetings_counter = Counter(flattened_attendance_list)

# Sort elements by their counts in descending order
amount_of_meetings_list = sorted(amount_of_meetings_counter.items(), key=lambda x: x[1], reverse=True)

# Inspect results
amount_of_meetings_list

In [ ]:
for member_name, count in amount_of_meetings_counter.items():
    # print(member_name, count)
    aanwezig_per_lid_alle_maar_niet_vast_df.at[member_name, "Aantal commissies extra aanwezig"] = count
    
# Inspect results
aanwezig_per_lid_alle_maar_niet_vast_df

In [34]:
def obtain_attendance_non_permanent_members(commissions_overview_df_input, 
                                            parlementsleden_all_dict_input,
                                            fracties_dict_input):
    """
    We obtain aggregated counts per member over all commissions in which they
    were no permanent member, to get overall totals per member how often they
    had a certain attendance     status (e.g. 'Aanwezig', 'Afwezig', 'Verontschuldigd'), 
    using a helper function `obtain_aggregated_counts()`.
    """
    # Obtaining actual aggregated counts for how often members were present
    aanwezig_per_lid_alle = obtain_aggregated_counts(commissions_overview_df_input['aanwezig_count_alle'])
    # Set member names as index (to allow easier processing later on)
    aanwezig_per_lid_alle = aanwezig_per_lid_alle.set_index("Member")
    
    
    # Then we obtain a list of all members of parliament, to compare them with the attendance data.
    alle_parlementsleden_list = [parlementsleden_all_dict_input[member_tuple][0] for member_tuple in parlementsleden_all_dict_input.keys()]
    
    
    # Obtain list of all members of parliament that have never attended a commission
    all_members_never_present = [element for element in alle_parlementsleden_list if element not in aanwezig_per_lid_alle.index]

    
    # Then we obtain for each commission a list of members that have attended meetings of that commission although they are no permanent member of that commission.
    members_present_but_no_permanent_overall = [] # Create empty list to fill

    # Iterate over each commission
    for index_overview, row_overview in commissions_overview_df_input.iterrows():
        permanent_members = row_overview["vaste leden"] # Obtain permanent members of commission
        # Obtain list of all members that have been present at at least 1 meeting of this commission
        members_present_in_meetings = [member_tuple[0] for member_tuple in row_overview["aanwezig_count_alle"]]
        # Obtain subset of those members that are no permanent members
        members_present_but_no_permanent = [member for member in members_present_in_meetings if member not in permanent_members]
        # Append this subset to the overall list
        members_present_but_no_permanent_overall.append(members_present_but_no_permanent)
      
    
    # Then we obtain for each of those non-permanent members how often they were present at a specific commission.
    count_members_present_but_not_permanent_overall = []
    for index, spec_commission_attendance_list in enumerate(commissions_overview_df_input['aanwezig_count_alle']):
        count_members_present_but_not_permanent = [member_tuple for member_tuple in spec_commission_attendance_list if member_tuple[0] in members_present_but_no_permanent_overall[index]]
        count_members_present_but_not_permanent_overall.append(count_members_present_but_not_permanent)
    # Append resulting list as new column to overview data frame 
    commissions_overview_df_input["aanwezig_count_alle_maar_geen_vast_lid"] = count_members_present_but_not_permanent_overall
        
        
    # Obtaining actual aggregated counts for how often members were present
    aanwezig_per_lid_alle_maar_niet_vast_df = obtain_aggregated_counts(commissions_overview_df_input['aanwezig_count_alle_maar_geen_vast_lid'])
    # Use member name as index
    aanwezig_per_lid_alle_maar_niet_vast_df = aanwezig_per_lid_alle_maar_niet_vast_df.set_index('Member')

    
    # Then we add the party the dataframe, using the earlier defined helper function.
    parties_list = []
    for member in aanwezig_per_lid_alle_maar_niet_vast_df.index:
        party = find_member(fracties_dict_input, member)
        parties_list.append(party)
    aanwezig_per_lid_alle_maar_niet_vast_df["Partij"] = parties_list
    
    
    # We also add in how many additional commissions they were present.
    aanwezig_per_lid_alle_maar_niet_vast_df["Aantal commissies extra aanwezig"] = 0
    # Flatten list of attendance (including counts)
    flattened_attendance_list = [item for sublist in commissions_overview_df_input["aanwezig_count_alle_maar_geen_vast_lid"] for item in sublist]
    # Maintain only names and not counts
    flattened_attendance_list = [item[0] for item in flattened_attendance_list]
    # Count occurrences of members for all meetings represented in attandance_list
    amount_of_meetings_counter = Counter(flattened_attendance_list)
    # Sort elements by their counts in descending order
    amount_of_meetings_list = sorted(amount_of_meetings_counter.items(), key=lambda x: x[1], reverse=True)
    for member_name, count in amount_of_meetings_counter.items():
    aanwezig_per_lid_alle_maar_niet_vast_df.at[member_name, "Aantal commissies extra aanwezig"] = count
    
    
    # Finally we also add for each member in how many commissions they are a permanent member.
    aanwezig_per_lid_alle_maar_niet_vast_df["Aantal commissies waarin vast lid"] = 0 # Initialize new column
    # Create flattened list of all permanent member accross all commissions
    flattened_permanent_members = [item for sublist in commissions_overview_df_input['vaste leden'] for item in sublist]
    # Count occurrences of meetings for which member is permanent member
    amount_of_meetings_as_permanent_counter = Counter(flattened_permanent_members)
    #Store counts in column '"Aantal commissies waarin vast lid"'
    for member_name, count in amount_of_meetings_as_permanent_counter.items():
        if member_name in aanwezig_per_lid_alle_maar_niet_vast_df.index:
            aanwezig_per_lid_alle_maar_niet_vast_df.at[member_name, "Aantal commissies waarin vast lid"] = count
            
    return aanwezig_per_lid_alle_maar_niet_vast_df

Finally we also add for each member in how many commissions they are a permanent member. 

In [38]:
# Initialize new column
aanwezig_per_lid_alle_maar_niet_vast_df["Aantal commissies waarin vast lid"] = 0

In [39]:
# Create flattened list of all permanent member accross all commissions
flattened_permanent_members = [item for sublist in commissions_overview_df['vaste leden'] for item in sublist]

# Count occurrences of meetings for which member is permanent member
amount_of_meetings_as_permanent_counter = Counter(flattened_permanent_members)

#Store counts in column '"Aantal commissies waarin vast lid"'
for member_name, count in amount_of_meetings_as_permanent_counter.items():
    if member_name in aanwezig_per_lid_alle_maar_niet_vast_df.index:
        aanwezig_per_lid_alle_maar_niet_vast_df.at[member_name, "Aantal commissies waarin vast lid"] = count
    
# Inspect results
aanwezig_per_lid_alle_maar_niet_vast_df

,Aggregated_Count,Partij,Aantal commissies waarin vast lid
Member,,,
Thijs Verbeurgt,18,Vooruit,1
Stijn De Roo,18,cd&v,2
Jeroen Tiebout,16,N-VA,2
Steve Vandenberghe,15,Vooruit,1
Johan Danen,15,Groen,1
...,...,...,...
Els Sterckx,1,Vlaams Belang,2
Carmen Ryheul,1,Vlaams Belang,2
Suzy Wouters,1,Vlaams Belang,2


In [40]:
print("Volgende parlementsleden daagden het meest op in commissies waar zij geen vast lid van zijn:")
aanwezig_per_lid_alle_maar_niet_vast_df.head()
print("Volgende parlementsleden daagden het minst op in commissies waar zij geen vast lid van zijn:")
aanwezig_per_lid_alle_maar_niet_vast_df.tail()

Volgende parlementsleden daagden het meest op in commissies waar zij geen vast lid van zijn:


,Aggregated_Count,Partij,Aantal commissies waarin vast lid
Member,,,
Thijs Verbeurgt,18,Vooruit,1
Stijn De Roo,18,cd&v,2
Jeroen Tiebout,16,N-VA,2
Steve Vandenberghe,15,Vooruit,1
Johan Danen,15,Groen,1


Volgende parlementsleden daagden het minst op in commissies waar zij geen vast lid van zijn:


,Aggregated_Count,Partij,Aantal commissies waarin vast lid
Member,,,
Els Sterckx,1,Vlaams Belang,2
Carmen Ryheul,1,Vlaams Belang,2
Suzy Wouters,1,Vlaams Belang,2
Jos D'Haese,1,PVDA,9
Sihame El Kaouakibi,1,Onafhankelijke,1


In [ ]:
#Export results
aanwezig_per_lid_alle_maar_niet_vast_df.to_csv(f'../data/attendance_non_permanent_members_{today_str}.csv', sep=';')
aanwezig_per_lid_alle_maar_niet_vast_df.to_pickle(f'../data/attendance_non_permanent_members_{today_str}.pkl')

# Test chunks

In [ ]:
set(meetings_all_commissions_filtered_df["commissie.titel"])

In [ ]:
# for index_overview, row_overview in commissions_overview_filtered_df.iterrows():
#     print(row_overview["commissie.titel"])
# meetings_all_commissions_filtered_df

In [ ]:
set(meetings_all_commissions_filtered_df['commissie.titel'])

In [ ]:
meetings_all_commissions_filtered_df[
        meetings_all_commissions_filtered_df["commissie.titel"] == 'Commissie voor de bestrijding van gewelddadige radicalisering']

In [ ]:
meetings_all_commissions_filtered_df_relevant['AANWEZIG']

obtain_attendance_counter(meetings_all_commissions_filtered_df_relevant['AANWEZIG'])

In [ ]:
attendance_list

In [ ]:
meetings_all_commissions_filtered_df_relevant[column_name_spec]

[item["Naam"] for item in meetings_all_commissions_filtered_df_relevant[column_name_spec] if isinstance(item, list)]

In [ ]:
# Inspect results
commissions_overview_filtered_df

In [ ]:
for index_spec, row_specific in meetings_all_commissions_filtered_df.iterrows():

In [ ]:
# get_count(meetings_all_commissions_filtered_df.loc["Vergadering 1764019", 'AFWEZIG'])

In [ ]:
commissions_overview_filtered_df
meetings_all_commissions_filtered_df

In [ ]:
meetings_all_commissions_filtered_df.columns

In [ ]:
for index_spec, row_spec in 

In [ ]:
flattened_list_attendance = []
for attendance_list in meetings_all_commissions_filtered_df["AFWEZIG_vast"]:
    attendance_names = [item["Naam"] for item in attendance_list]
    flattened_list_attendance.extend(attendance_names)
flattened_list_attendance  

flattened_list_attendance_counter = Counter(flattened_list_attendance)
flattened_list_attendance_counter